In [1]:
from custom_model.models import DGCN_vis, SoftLoss, DERLoss
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

import torch
from torch.optim.lr_scheduler import StepLR

from custom_model.utils import *
from custom_model.training import *
from custom_model.custom_dataset import *
from custom_model.visual_utils import *

from absl import logging
logging._warn_preinit_stderr = 0
logging.warning('Worrying Stuff')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

Using cuda device


In [2]:
para = {}
para['time_invertal'] = 5
para['horizon'] = 15
para['observation'] = 20
para['nb_node'] = 193
para['dim_feature'] = 128
A = adjacency_matrix(3)
B = adjacency_matrixq(3, 8)

Av, Aq = get_directed_connection()
links, lengths, indicators = get_roads()

model = DGCN_vis(para, A, B, return_interpret=True).to(device)
model.load_state_dict(torch.load('./pretrained/predictor_uq.pt'))
testset = AMSdataset(['2019'], para, 'test')

v_mean = np.mean(testset.X[:,-15:,:,0], (1,2))
order = np.argsort(v_mean)

preprocessing data...
completed


In [ ]:
D = []
for i in tqdm(range(len(testset))):
    _, _, _, _, _, _, _, demand = get_prediction(model, testset, i)
    D.append(demand)
D = np.array(D)
print(D.shape)

In [ ]:
groundtruth, speed, flow, alea, epis, m1, m2, demand = get_prediction(model, testset, order[4])

In [ ]:
fig = plot_prediction(groundtruth, flow, mode='flow')

In [ ]:
fig.savefig('./imgs/q_spatial_ff19.jpg', dpi=600)

In [ ]:
fig = plot_state_on_map(links, speed[7], mode='speed')

In [ ]:
fig2 = plot_state_on_map(links, flow[10], mode='flow')

In [ ]:
fig2 = plot_demand_on_map(links, demand[0]*30, 400)

In [ ]:
fig = draw_interpretation(links, Av, m1[7], 19, 'speed', clear_axis=True)

In [ ]:
fig = draw_interpretation(links, Aq, m2[7], 19, 'flow', colorbar=True, clear_axis=True, show_lines=False, ylim=[52.37, 52.44])

In [ ]:
fig = plot_prediction(groundtruth, speed, mode='speed')

In [ ]:
fig.savefig('./imgs/example_q2.pdf', dpi=600)

In [ ]:
plt.scatter(np.arange(193), speed[10], c=speed[10], vmin=0, vmax=130, cmap=cm.rainbow_r)

In [1]:
epis.shape

NameError: name 'epis' is not defined

In [ ]:
plt.plot(np.mean(D, (0,1)))